# 🧪 Notebook 2 — TaskFlow API with Spring Boot (Phase 0, hands‑on)

> **Goal:** run the **tasks-service** (Phase 0), validate `/actuator/health`, and exercise the MVP endpoints **without the gateway**.  
> **Theory→Practice bridge:** throughout the notebook you’ll find **Link Notes** connecting the steps to the **Twelve‑Factor App** principles.

---

## 🎯 What you will achieve

- Start the **tasks-service** locally (via **STS** or **Maven**; **Docker** optional).
- Confirm it responds on **http://localhost:8081**.
- Exercise endpoints: **POST /tasks**, **GET /tasks**, **GET /tasks/{id}**, **PATCH /tasks/{id}/status**.
- Verify **observability**: `GET /actuator/health` returns **UP**.
- Organize tests with **Postman** (or `curl`).

---

## 🧭 How this notebook maps to the 12 Factors

| Practical action | Factors touched |
|---|---|
| Clone repo and use phase tags | **#1 Codebase** |
| Start service (STS/Maven/Docker) | **#5 Build/Release/Run**, **#7 Port Binding**, **#9 Disposability** |
| Use H2 in dev | **#4 Backing services**, **#6 Processes (stateless)** |
| Expose `/actuator/health` | **#7 Port Binding**, **#9 Disposability** |
| Test with Postman/`curl` | **#7 Port Binding**, **#11 Logs** |
| Tweak via profiles/env vars (when used) | **#3 Config**, **#10 Dev/Prod parity** |

You’ll see **🔗 Link Note (12‑Factor …)** callouts inline.

---

## 🧰 Prerequisites

- **Java 21**, **Maven**, **Git**
- **STS (Spring Tool Suite)** (or another IDE)
- **Postman** (recommended) — alternative: `curl`
- **Docker** (optional, if you prefer containers)

---

## 1) Get the code (Phase 0)

```bash
git clone https://github.com/smartlearningci/cloud_java
cd cloud_java
git checkout phase-0
```

**What’s here (Phase 0):** `tasks-service` using Spring Boot + H2 + Actuator + MVP endpoints.

🔗 **Link Note (12‑Factor #1 Codebase)**  
Single, clear codebase per service and **tags** for stable states (phase‑0) enable multiple deploys from the same source.

---

## 2) Run the service (pick one)

### Option A — STS (recommended for development)
1. **File → Import… → Maven → Existing Maven Projects** → select the project folder.  
2. Open the main class (e.g., `…TasksServiceApplication`).  
3. **Run As → Spring Boot App**.  
4. Check the console: the app should listen on **port 8081**.

### Option B — Maven from terminal
```bash
# inside the tasks-service project
./run_local.sh
# (fallback if the script is not present)
# mvn spring-boot:run
```

### Option C — Docker (optional)
```bash
# inside the tasks-service project
./run_docker.sh
# the service should be available on http://localhost:8081 (per script/Dockerfile mapping)
```

🔗 **Link Note (12‑Factor #5 Build/Release/Run, #7 Port Binding, #9 Disposability)**  
You keep build separate from run (scripts/image), the app **exports HTTP** on port 8081, and **starts/stops quickly** (useful for rapid feedback).

---

## 3) Minimal observability (Actuator)

- **Health:** open `http://localhost:8081/actuator/health` → it should return `{"status":"UP"}`.  
- **Logs:** watch the startup in the STS/terminal console.  
- **H2 Console** (if enabled in the project): `http://localhost:8081/h2-console`  
  - **Note:** H2 is **in‑memory** here, so data **does not** persist across restarts.

🔗 **Link Note (12‑Factor #7 Port Binding, #9 Disposability, #11 Logs)**  
HTTP health makes the service “observable” and orchestratable; logs flow to stdout/stderr (streams).

---

## 4) Tests with **Postman** (recommended)

### 4.1 Environment
- `baseUrl = http://localhost:8081`

### 4.2 Suggested requests (copy & run)

1) **Health**  
- **GET** `{{baseUrl}}/actuator/health` → `UP`

2) **List tasks**  
- **GET** `{{baseUrl}}/tasks` → `200 OK` with `[]` (or seeded content, if present)

3) **Create task**  
- **POST** `{{baseUrl}}/tasks`  
  Headers: `Content-Type: application/json`  
  Body:
```json
{
  "title": "Write documentation",
  "description": "Notebook 2 - Phase 0",
  "projectId": "DOCS",
  "assignee": "Ana"
}
```
- Expected: `201 Created` + **Task** body

4) **Get by ID**  
- **GET** `{{baseUrl}}/tasks/{{taskId}}` → `200 OK` or `404 Not Found`

5) **Update status**  
- **PATCH** `{{baseUrl}}/tasks/{{taskId}}/status`  
  Headers: `Content-Type: application/json`  
  Body:
```json
{ "status": "DOING" }
```
- Expected: `200 OK` + updated task (then try `DONE`)

6) **List with filters**  
- **GET** `{{baseUrl}}/tasks?status=DOING&projectId=DOCS` → `200 OK`

🔗 **Link Note (12‑Factor #7 Port Binding, #11 Logs)**  
You’re talking **directly** to the app on port 8081; errors/info are visible in the service’s **stdout**.

---

## 5) `curl` alternative (if you prefer terminal)

```bash
# health
curl -s http://localhost:8081/actuator/health

# list
curl -s http://localhost:8081/tasks | jq

# create
curl -s -H "Content-Type: application/json"   -d '{"title":"Write documentation","description":"Notebook 2 - Phase 0","projectId":"DOCS","assignee":"Ana"}'   -X POST http://localhost:8081/tasks | jq

# get by id
curl -s http://localhost:8081/tasks/<ID> | jq

# update status
curl -s -H "Content-Type: application/json"   -d '{"status":"DOING"}'   -X PATCH http://localhost:8081/tasks/<ID>/status | jq
```

If you don’t have `jq`, remove the `| jq` beautifier.

---

## 6) Validations & errors (observe responses)

- **`title` required**: try creating without `title` → expect **400 Bad Request** (JSON error payload).  
- **Non‑existing ID**: `GET/PATCH` with an invalid id → **404 Not Found**.  
- **Invalid transition**: attempting a disallowed state change → **4xx** (per service policy).

🔗 **Link Note (12‑Factor #3 Config, #10 Dev/Prod parity)**  
Consistent validation and error payloads help keep environments aligned and diagnosis fast.

---

## 7) Quick troubleshooting

- **Port 8081 busy**: stop previous instances or adjust the local profile/port.  
- **Java 21**: check `java -version` and STS JDK setup.  
- **Maven**: run `mvn -U clean package` or **Project → Update** in STS.  
- **Docker**: ensure the Docker service is running; on Linux, add your user to `docker` group and re‑login.  
- **Postman**: `Content-Type: application/json` and valid JSON for `POST/PATCH`.

🔗 **Link Note (12‑Factor #9 Disposability, #10 Dev/Prod parity)**  
Fast start/stop and predictable, similar environments accelerate feedback and reduce “works on my machine” issues.

---

## ✅ Notebook 2 checkpoints

- [ ] I can **start the tasks-service** (STS or Maven/Docker) and it listens on **8081**.  
- [ ] **/actuator/health** returns **UP**.  
- [ ] I can **create → list → get by id → update status**.  
- [ ] I understand that **H2** is **in‑memory** (data resets on restart).  
- [ ] I’ve saved a **Postman collection** (or commands) for reuse.

**Most‑touched 12‑Factors in this notebook:** **#1, #4, #5, #6, #7, #9, #10, #11**.

---

## 🔜 Next

**Notebook 3 — Gateway + X‑API‑KEY Security (Phase 1)**  
- Put the **Gateway** in front of `tasks-service`.  
- Enable and validate **`X-API-KEY`** (environment variable in the gateway).  
- Repeat the tests via `/api/tasks/**` (**single entry point**).
